<center><img src="img/logo_hse_black.jpg"></center>

<h1><center>Методы машинного обучения</center></h1>
<h2><center>Введение в анализ сетевых структур: (Social) Network Analysis</center></h2>

In [ ]:
%matplotlib inline

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

plt.style.use('ggplot')
plt.rcParams['figure.figsize'] = (12,8)

C сетями мы будем работать с помощью модуля [NetworkX](https://networkx.github.io/documentation/stable/). Функционала в нем постеменно становится больше и больше, но по скорости работы он сильно уступает библиотекам, реализованным на С и С++

In [ ]:
import networkx as nx

## Создадим граф

In [ ]:
g = nx.Graph()

In [ ]:
g

In [ ]:
# Как корабль назовешь..
g.graph['some title'] = 'first graph'

In [ ]:
g.graph

## Добавим вершины

In [ ]:
g.add_node(1)

In [ ]:
g.number_of_nodes()

In [ ]:
g.add_node('some_node') # можно создавать вершины с произвольным* идентификатором

## Добавим ребра

In [ ]:
g.add_edge(0, 1) # если вершины с идентификатором х еще не было, то она автоматически сгенерируется

In [ ]:
g.edges()

In [ ]:
g.nodes()

In [ ]:
g.add_edges_from([(0,2), (1,3), (4,3), (1,2), (2,2), (3,2)])

In [ ]:
g.edges()

## Различные операции с вершинами и ребрами

In [ ]:
g.degree() # Степени вершин (количество связей для каждой вершины)

In [ ]:
g.degree['some_node']

Вершинам можно задавать различные атрибуты (читай "признаки")

In [ ]:
for n_id in g.nodes():
    g.node[n_id]['label'] = 'v_{}'.format(n_id)

In [ ]:
nx.get_node_attributes(g, 'label')

In [ ]:
g.nodes[1]

In [ ]:
g.nodes.data()

Иногда это удобнее делать так:

In [ ]:
some_attributes = {0: 'val1', 1: 'val2'}
nx.set_node_attributes(g, some_attributes, 'attr')

In [ ]:
g.nodes.data()

Тоже самое для ребер

In [ ]:
g.edges[1,0]['type'] = 'friendship'

In [ ]:
g.edges[1,0]

In [ ]:
# проверка различных свойств графа
g.is_directed() 

In [ ]:
nx.is_connected(g)

## Объединение графов

In [ ]:
g1 = g.copy()
g2 = nx.erdos_renyi_graph(10, 0.3)

In [ ]:
g3 = nx.disjoint_union(g1, g2)
# g3 = nx.union(g1, g2, rename=('g1_', 'g2_'))

In [ ]:
print(g1)
print('=' * 10)
print(g2)

In [ ]:
print(g1.number_of_nodes())
print(g2.number_of_nodes())
print(g3.number_of_nodes())

In [ ]:
g3.nodes()

Создадите граф с рисунка

<img src='./img/clique_init.png' width="550"/>

In [ ]:
g = nx.complete_graph(6)

g.add_nodes_from(range(6,10))
g.add_edges_from([(6,8), (6,7), (7,8), (8,9), (9,7)])

g.add_edges_from([(2,6), (1,7), (5,9)])

In [ ]:
nx.draw(g)

Выведем матрицу смежности, список смежности и список ребер этого графа

In [ ]:
A = nx.adj_matrix(g)
A

In [ ]:
A.todense()

In [ ]:
nx.adjacency_data(g)

Сохраним что-нибудь из этого:

In [ ]:
nx.write_edgelist(g, 'graph.edglist')
nx.write_graphml(g, 'graph.gml')

In [ ]:
!head graph.gml # на Windows не сработает

In [ ]:
!head graph.edglist # на Windows не сработает

## (ваши) Соц-графы

* [Туториал](https://nbviewer.jupyter.org/github/allatambov/Py-programming-3/blob/master/15-06/lect-vk-api.ipynb) по выгрузке данных с помощью VKAPI

In [ ]:
import json
import os

In [ ]:
vkid = '1504653'
filepath = os.path.join('data', vkid, 'friends_{}.json'.format(vkid))

with open(filepath, 'r') as fin:
    friends_temp = json.load(fin)[0]

In [ ]:
# Словарь с вашими друзьями
friends_temp

In [ ]:
# Просто меняем строковый ключ на числовой
friends = dict()
for k in friends_temp.keys():
    friends[int(k)] = friends_temp[k]

In [ ]:
filepath = os.path.join('data', vkid, 'common_friends_{}.json'.format(vkid))

with open(filepath, 'r') as fin:
    common_friends_temp = json.load(fin)

In [ ]:
# Словарь с общими друзьями между вами и вашими друзьями
common_friends_temp

In [ ]:
# Просто меняем строковый ключ на числовой
common_friends = dict()
for k in common_friends_temp.keys():
    common_friends[int(k)] = common_friends_temp[k]

In [ ]:
g = nx.Graph()

In [ ]:
friends

In [ ]:
# Делаем список из пар (vkid, словарь с характеристиками)

g.add_nodes_from((fid, {'sex': friend.get('sex', -1), 
                        'first_name': friend.get('first_name', -1), 
                        'last_name': friend.get('last_name', -1), 
                        'university_name': friend.get('university_name', -1)}) for fid, friend in friends.items())

In [ ]:
g.number_of_nodes()

In [ ]:
# Делаем список из пар (vkid1, vkid2)
g.add_edges_from((f1, f2) for f1, f_list in common_friends.items() for f2 in f_list)

In [ ]:
g.number_of_edges()

In [ ]:
nx.draw_kamada_kawai(g)

In [ ]:
nx.write_graphml(g, 'vk_graph.graphml')

Попробуем открыть это дело в gephi!

# Характеристики вершин/ребер сети

На основе сетевой структуры для вершин и ребер можно расчитать характеристики "важности" этих элементов.

Они еще называются "центральностями"

In [ ]:
g_nx = nx.karate_club_graph()

In [ ]:
g_nx.number_of_edges()

In [ ]:
g_nx.number_of_nodes()

In [ ]:
nx.draw(g_nx)

## Degree centrality

Самая оцевидная центральность - просто степень узла. Характеризует некоторую популярность узла (много друзей, много связей).

$$ C_d(i) = k(i) = \sum_jA_{ij} = \sum_iA_{ij}$$
$$ \bar{C}_d(i) = \frac{1}{n-1} C_d(i)$$

Существует обобщение на ориентированные (prestige) и взвешенные сети.

In [ ]:
degr = g_nx.degree()
degr_cent = nx.centrality.degree_centrality(g_nx)

In [ ]:
degr = np.array(degr) * 100
nx.draw_networkx(g_nx, node_size=degr)

## Closeness centrality

Центральность, основанная на расстоянии до остальных вершин в графе.

$$ C_{cl}(i) = \frac{1}{\sum_j d(i,j)} $$

$$ \bar{C}_{cl}(i) = (n-1) \cdot C_{cl}(i) $$

Актор, расположенный в центре сети может быстро добраться до остальных акторов. Акторы на периферии расположены дальше.

**Вопрос:** что будет, если граф окажется несвязным?

In [ ]:
closeness_nodes = nx.centrality.closeness_centrality(g_nx)

In [ ]:
closeness = np.array(list(closeness_nodes.values())) * 1000
nx.draw_networkx(g_nx, node_size=closeness)

## Betweenness centrality (nodes)

Пусть $\sigma_{st}$ - количество кратчайших путей между вершинами $s$ и $t$, а $\sigma_{st}(i)$ - кр. пути между $v_s$ и $v_t$, которые проходят через вершину $v_i$.

Тогда 
$$ C_b(i) = \sum\limits_{s\neq t\neq i} \frac{\sigma_{st}(i)}{\sigma_{st}} $$

$$ \bar{C}_b(i) = \frac{2}{(n-1)(n-2)}C_b(i) $$

In [ ]:
betw_nodes = nx.betweenness_centrality(g_nx) 

betw = np.array(list(betw_nodes.values())) * 1000
nx.draw_networkx(g_nx, node_size=betw)

## Betweenness centrality (edges)

Betweenness также можно расчитывать для ребер! Давайте определим для каких ребер она наибольшая и что это может нам дать?

In [ ]:
betw_edg = nx.centrality.edge_betweenness_centrality(g_nx)

In [ ]:
sources = []
targets = []
betw = []
for e, b in betw_edg.items():
    sources.append(e[0])
    targets.append(e[1])
    betw.append(b)
df = pd.DataFrame({'source': sources, 
                   'target': targets,
                   'betw': betw})

In [ ]:
df.sort_values('betw', ascending=False).head()

## Page Rank

Идея PageRank заключается в попытке описать блуждание по вершинам графа. Вероятность перехода в вершину $v_i$ обратнопропорциональна степеням входящих связанных с ней вершин.

$$p^{t+1} = (D^{-1}A)^\top p^t = P^\top p^t$$

Помимо случайного блуждания между соседними вершинами заложен механизм "телепорта" между случайными вершинами с вероятностью $1-\alpha$.

$$ \mathbb{P} = \alpha P + \frac{(1 - \alpha)}{n} E,$$
где $E$ - это матрица состоящая из единиц.

Аналогичным образом решается задача на поиск собственного числа

$$\mathbb{P}^\top p = \lambda p$$

In [ ]:
pr_nodes = nx.pagerank(g_nx)
pr = np.array(list(pr_nodes.values())) * 1500

nx.draw_networkx(g_nx, node_size=pr)

## "Геометрическая" центральность

Eccentricity - максимальная длина кратчайшего пути из вершины $i$ до всех остальных вершин $e(i) = \max\limits_j d(i, j)$.

Диаметр - $\max e(i)$<br/>
Радиус - $\min e(i)$

Центральными вершинами являются те, у которых $e(i)$ равна радиусу графа

In [ ]:
print(nx.radius(g_nx))
print(nx.diameter(g_nx))

In [ ]:
ecc_nodes = nx.eccentricity(g_nx)
ecc = np.array(list(ecc_nodes.values())) * 100

nx.draw_networkx(g_nx, node_size=ecc)

### Clustering coefficient

Доля "треугольников" в окресности вершины.

In [ ]:
# nx.transitivity(g_nx)
nx.triangles(g_nx)